In [ ]:
from IPython.core.display import HTML
with open('../style.css', 'r') as file:
    css = file.read()
HTML(css)

# Converting a Non-Deterministic <span style="font-variant:small-caps;">Fsm</span> into a Deterministic <span style="font-variant:small-caps;">Fsm</span>

In this notebook we show how a non-deterministic <span style="font-variant:small-caps;">Fsm</span>
$$ F = \langle Q, \Sigma, \delta, q_0, A \rangle $$
can be transformed into a deterministic <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(F)$ such that both <span style="font-variant:small-caps;">Fsm</span>s accept the
same language, that is we have
$$ L(F) = L\bigl(\texttt{det}(F)\bigr). $$
The idea behind this transformation is that the <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(F)$ has to 
compute the set of all states that the <span style="font-variant:small-caps;">Fsm</span> $F$ could be in. 
Hence the states of the deterministic <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(F)$ are 
**sets** of states of the non-deterministic <span style="font-variant:small-caps;">Fsm</span> $F$.  A set of these states contains all those states that the non-deterministic <span style="font-variant:small-caps;">Fsm</span> 
$F$ could have reached.  Furthermore, a set $M$ of states of the <span style="font-variant:small-caps;">Fsm</span> $F$ is an accepting state of the <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(F)$ if the set $M$ contains an accepting state of the <span style="font-variant:small-caps;">Fsm</span> $F$.

<hr style="height:5px;background-color:blue">

In order to present the construction of $\texttt{det}(F)$ we first have to define two auxiliary functions.
We start with the <em style="color:blue">$\varepsilon$-closure</em> of a given state. 

The function `epsClosure` takes two arguments:
- `s` is a state, 
- `delta` is the transition function of the non-deterministic <span style="font-variant:small-caps;">Fsm</span> $F$.

The function computes the set of all those states that can be reached from the state
`s` via $\varepsilon$-transitions.
Formally, the set $\texttt{epsClosure}(q)$ is defined inductively:
- $s \in \texttt{epsClosure}(s)$.
- $p \in \texttt{epsClosure}(s) \wedge r \in \delta(p, \varepsilon) \;\rightarrow\; r \in \texttt{epsClosure}(s)$.
 
  If the state $p$ is an element of the $\varepsilon$-closure of the state $s$ 
  and there is an $\varepsilon$-transition from $p$ to some state $r$, then $r$ 
  is also an element of the $\varepsilon$-transition of $s$.

In [ ]:
def epsClosure(s, delta):
    Result = { s }
    while True:
        NewStates = { p for q in Result 
                        for p in delta.get((q, ''), set())
                    }
        if NewStates <= Result:
            return frozenset(Result)
        Result |= NewStates

In order to transform a non-deterministic <span style="font-variant:small-caps;">Fsm</span> $F$ into a deterministic 
<span style="font-variant:small-caps;">Fsm</span>
$\texttt{det}(F)$ we have to extend the function $\delta:Q \times \Sigma \rightarrow 2^Q$ into the function
$$\delta^*: Q \times \Sigma \rightarrow 2^Q. $$
The idea is that given a state $q$ and a character $c$,  the value of $\delta^*(q,c)$ is the set of all states that the
<span style="font-variant:small-caps;">Fsm</span> $F$ could reach when it reads the character $c$ in state $q$ and then performs an arbitrary number of $\varepsilon$-transitions.  Formally, the definition of $\delta^*$ is as follows:
$$ \delta^*(q_1, c) := \bigcup \bigl\{ \texttt{epsClosure}(q_2) \bigm| q_2 \in \delta(q_1, c) \bigr \}. $$
This formula is to be read as follows:
- For every state $q_2 \in Q$ that can be reached from the state $q_1$ by reading the character $c$ we
  compute $\texttt{epsClosure}(q_2)$.
- Then we take the union of all these sets $\texttt{epsClosure}(q_2)$.

The function $\delta^*$ is implemented as the function `deltaStar`, which takes three arguments:
- `s` is a state,
- `c` is a character,
- `𝛿` is the transition function of a deterministic 
  <span style="font-variant:small-caps;">Fsm</span>.

This function computes the set of all those states that can be reached 
from `s` when we first have a transition from state `s` to some state `p` 
on reading the character `c` followed by any number of $\varepsilon$-transitions
starting in `p`.

In [ ]:
def deltaStar(s, c, delta):
    return { p for q in delta.get((s, c), set())
               for p in epsClosure(q, delta)
           }

The function  $\delta^*$ maps a state into a set of states.  Since the <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(F)$ uses sets of states of the <span style="font-variant:small-caps;">Fsm</span> $F$ as its states we need a function that maps sets of states of the <span style="font-variant:small-caps;">Fsm</span> $F$ into sets of states.  Hence we generalize 
the function $\delta^*$ to the function
$$ \Delta: 2^Q \times \Sigma \rightarrow 2^Q $$
such that for a set $M$ of states and a character $c$ the expression $\Delta(M, c)$
computes the set of all those states that the <span style="font-variant:small-caps;">Fsm</span> $F$ could be in if it is in a state from the set $M$, then
reads the character $c$, and finally makes some $\varepsilon$-transitions.
The formal definition is as follows: 
$$ \Delta(M,c) := \bigcup \bigl\{ \delta^*(q,c) \bigm| q \in M \bigr\}. $$
This formula is easy to understand:  For every state  $q \in M$ we compute the set of states that the
<span style="font-variant:small-caps;">Fsm</span> could be in after reading the character $c$ and doing some 
$\varepsilon$-transitions.  Then we take the union of these sets.

In [ ]:
def capitalDelta(M, c, delta):
    Result = { s for q in M 
                 for s in deltaStar(q, c, delta) 
             }
    return frozenset(Result)

The function `allStates` takes three arguments:
- $Q$ is $\texttt{epsClosure}(q0)$, where $q_0$ is the start state of the deterministic <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(F)$,
- $\delta$ is the transition function of the non-deterministic <span style="font-variant:small-caps;">Fsm</span> $F$, and
- $\Sigma$ is the alphabet of the non-deterministic <span style="font-variant:small-caps;">Fsm</span> $F$.

The function `allStates` computes the set of all states of the deterministic <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(F)$
that can be reached from the start state.

In [ ]:
def allStates(Q, delta, Sigma):
    Result = { Q }
    while True:
        NewStates = { capitalDelta(M, c, delta) for M in Result 
                                                for c in Sigma 
                    }
        if NewStates <= Result:
            return Result
        Result |= NewStates

Now we are ready to formally define how the deterministic <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(F)$
is computed from the non-deterministic <span style="font-variant:small-caps;">Fsm</span>
$F = \bigl\langle Q, \Sigma, \delta, q_0, A \bigr\rangle$.
We define: 
$$ \texttt{det}(F) := \bigl\langle \texttt{allStates}(\texttt{epsClosure}(q_0)), \Sigma, \Delta, \texttt{epsClosure}(q_0), \widehat{A} \bigr\rangle $$
where the components of this tuple are given as follows:
- The set of states of $\texttt{det}(F)$ is the set of all states that can be reached from the set $\texttt{epsClosure}(q_0)$.
- The input alphabet $\Sigma$ does not change when going from $F$ to $\texttt{det}(F)$.
  After all, the deterministic <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(F)$ has to recognize the same language as the non-deterministic
  <span style="font-variant:small-caps;">Fsm</span> $F$.
- The function $\Delta$ that has been defined previously specified how the set of states change when a
  character is read.
- The start state $\texttt{epsClosure}(q_0)$ of the non-deterministic <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(F)$ is the set of all states
  that can be reached from the start state $q_0$ of the non-deterministic <span style="font-variant:small-caps;">Fsm</span> $F$
  via $\varepsilon$-transitions.
- The set of accepting states $\widehat{F}$ is the set of those subsets of $Q$ that contain an accepting
  state of the <span style="font-variant:small-caps;">Fsm</span> $Q$:
  $$\widehat{A} := \bigl\{ M \in 2^Q \mid M \cap A \not= \{\} \bigl\}. $$

In [ ]:
def nfa2dfa(nfa):
    States, Sigma, delta, q0, Final = nfa
    newStart   = epsClosure(q0, delta)
    NewStates  = allStates(newStart, delta, Sigma)
    newDelta   = { (M, c): capitalDelta(M, c, delta) for M in NewStates
                                                     for c in Sigma
                 }
    NewFinal   = { M for M in NewStates 
                     if  M & Final != set() 
                 }
    return NewStates, Sigma, newDelta, newStart, NewFinal

To test this function, use the notebook `Test-NFA-2-DFA.ipynb`.